<a href="https://colab.research.google.com/github/ghadaabbes/Cupcakes/blob/master/Ghada%20Abbes/Behavioural_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity dataset



*   Add column WakeUpTime
*   Add column sleep_duration -> optimizing the format
*   Some columns renamed as date -> datetime
*   Group By, it's used for a purpose ! getting the first measurement per day for each patient, which will be used then to calculate the difference between wakeUp time and first measurement. 



In [1]:
import pandas as pd
activity_data = pd.read_csv('all_activity.csv')

In [2]:
sleep_dataset = activity_data[activity_data['status'] == 'sleep']

In [3]:
sleep_dataset['date'] = pd.to_datetime(sleep_dataset['date'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [4]:
activity_data['duration'] = activity_data['duration'].astype(int)

In [5]:
activity_data['sleep_duration'] = pd.to_timedelta(sleep_dataset['duration'], unit='s')

In [6]:
activity_data['weekUpTime'] = sleep_dataset['date'] + pd.to_timedelta(activity_data['duration'], unit='s')

In [7]:
activity_data

,id,date,status,loc,duration,sleep_duration,weekUpTime
0,6,2021-05-01 00:10:03,sleep,master bedroom,6743,0 days 01:52:23,2021-05-01 02:02:26
1,6,2021-05-01 02:02:26,active,NaN,2255,NaT,NaT
2,6,2021-05-01 02:40:01,sleep,master bedroom,5026,0 days 01:23:46,2021-05-01 04:03:47
3,6,2021-05-01 04:03:47,active,NaN,2175,NaT,NaT
4,6,2021-05-01 04:40:02,sleep,master bedroom,6888,0 days 01:54:48,2021-05-01 06:34:50
...,...,...,...,...,...,...,...
15427,80,2021-08-18 15:25:02,still,kitchen,783,NaT,NaT
15428,80,2021-08-18 15:38:05,active,NaN,4616,NaT,NaT
15429,80,2021-08-18 16:55:01,still,kitchen,2233,NaT,NaT
15430,80,2021-08-18 17:32:14,active,NaN,3168,NaT,NaT


In [8]:
activity_data['date'] = pd.to_datetime(activity_data['date'])
activity_data['Time'] = activity_data['date'].dt.time
activity_data['Date'] = activity_data['date'].dt.date

In [9]:
activity_data_groupedByDate = (pd
     .DataFrame(activity_data)#.agg({'date' : [np.min, np.max]})
    .groupby(['id',	'Date'])
    .apply(lambda gdf: (gdf
    .assign(sleep_duration=lambda sleep_duration: pd.to_timedelta(activity_data['duration'], unit='s'))))) 

In [10]:
activity_data_groupedByDate[1400:1520]

id                date  ...      Time        Date
id Date                                     ...                      
6  2021-06-30 1400   6 2021-06-30 22:13:56  ...  22:13:56  2021-06-30
   2021-07-01 1401   6 2021-07-01 00:10:03  ...  00:10:03  2021-07-01
              1402   6 2021-07-01 00:12:20  ...  00:12:20  2021-07-01
              1403   6 2021-07-01 00:40:02  ...  00:40:02  2021-07-01
              1404   6 2021-07-01 01:01:09  ...  01:01:09  2021-07-01
...                 ..                 ...  ...       ...         ...
   2021-07-07 1515   6 2021-07-07 18:55:02  ...  18:55:02  2021-07-07
              1516   6 2021-07-07 19:31:03  ...  19:31:03  2021-07-07
              1517   6 2021-07-07 20:55:02  ...  20:55:02  2021-07-07
              1518   6 2021-07-07 22:00:59  ...  22:00:59  2021-07-07
              1519   6 2021-07-07 22:40:02  ...  22:40:02  2021-07-07

[120 rows x 9 columns]

Trying to optimise sleep duration format

In [11]:
from string import Formatter
from datetime import timedelta

def strfdelta(tdelta, fmt='{D:02}d {H:02}h {M:02}m {S:02}s', inputtype='timedelta'):
    """Convert a datetime.timedelta object or a regular number to a custom-
    formatted string, just like the stftime() method does for datetime.datetime
    objects.

    The fmt argument allows custom formatting to be specified.  Fields can 
    include seconds, minutes, hours, days, and weeks.  Each field is optional.

    Some examples:
        '{D:02}d {H:02}h {M:02}m {S:02}s' --> '05d 08h 04m 02s' (default)
        '{W}w {D}d {H}:{M:02}:{S:02}'     --> '4w 5d 8:04:02'
        '{D:2}d {H:2}:{M:02}:{S:02}'      --> ' 5d  8:04:02'
        '{H}h {S}s'                       --> '72h 800s'

    The inputtype argument allows tdelta to be a regular number instead of the  
    default, which is a datetime.timedelta object.  Valid inputtype strings: 
        's', 'seconds', 
        'm', 'minutes', 
        'h', 'hours', 
        'd', 'days', 
        'w', 'weeks'
    """

    # Convert tdelta to integer seconds.
    if inputtype == 'timedelta':
        remainder = int(tdelta.total_seconds())
    elif inputtype in ['s', 'seconds']:
        remainder = int(tdelta)
    elif inputtype in ['m', 'minutes']:
        remainder = int(tdelta)*60
    elif inputtype in ['h', 'hours']:
        remainder = int(tdelta)*3600
    elif inputtype in ['d', 'days']:
        remainder = int(tdelta)*86400
    elif inputtype in ['w', 'weeks']:
        remainder = int(tdelta)*604800

    f = Formatter()
    desired_fields = [field_tuple[1] for field_tuple in f.parse(fmt)]
    possible_fields = ('W', 'D', 'H', 'M', 'S')
    constants = {'W': 604800, 'D': 86400, 'H': 3600, 'M': 60, 'S': 1}
    values = {}
    for field in possible_fields:
        if field in desired_fields and field in constants:
            values[field], remainder = divmod(remainder, constants[field])
    return f.format(fmt, **values)

In [12]:
activity_data_groupedByDate['sleep_duration'] = activity_data_groupedByDate.apply(lambda row : strfdelta(row['sleep_duration']) , axis = 1)


In [13]:
activity_data_groupedByDate.rename(columns={'date':'datetime'})

id            datetime  ...      Time        Date
id Date                                      ...                      
6  2021-05-01 0       6 2021-05-01 00:10:03  ...  00:10:03  2021-05-01
              1       6 2021-05-01 02:02:26  ...  02:02:26  2021-05-01
              2       6 2021-05-01 02:40:01  ...  02:40:01  2021-05-01
              3       6 2021-05-01 04:03:47  ...  04:03:47  2021-05-01
              4       6 2021-05-01 04:40:02  ...  04:40:02  2021-05-01
...                  ..                 ...  ...       ...         ...
80 2021-08-18 15427  80 2021-08-18 15:25:02  ...  15:25:02  2021-08-18
              15428  80 2021-08-18 15:38:05  ...  15:38:05  2021-08-18
              15429  80 2021-08-18 16:55:01  ...  16:55:01  2021-08-18
              15430  80 2021-08-18 17:32:14  ...  17:32:14  2021-08-18
              15431  80 2021-08-18 18:25:02  ...  18:25:02  2021-08-18

[15432 rows x 9 columns]

Problems: 

*   Redudance after group by: id and Date columns
*   How to delete the index 



# Weight dataset

I used the last dataset shared by Yasser, because I think it's cleaned then extracted columns just interested me !
Group By used to extract first weight measurement then I planned to join the dataset based on the same id and same Date (not datetime) to calculate the difference between the first measurement and WakeUp Time


In [14]:
weight_data = pd.read_csv('yasser_merged_DR.csv')

In [15]:
del weight_data['Unnamed: 0']

In [16]:
weight_data.reset_index(drop=True, inplace=True)

In [17]:
weight_data

,id,date,weight,WakeUp Time,toilet,meal,clothes,day,day_of_week,hour,is_weekend,season,active_1h_mean,active_2h_mean,active_3h_mean,active_4h_mean,active_5h_mean,active_6h_mean,away_1h_mean,away_2h_mean,away_3h_mean,away_4h_mean,away_5h_mean,away_6h_mean,sleep_1h_mean,sleep_2h_mean,sleep_3h_mean,sleep_4h_mean,sleep_5h_mean,sleep_6h_mean,still_1h_mean,still_2h_mean,still_3h_mean,still_4h_mean,still_5h_mean,still_6h_mean,score_1h_mean,score_2h_mean,score_3h_mean,score_4h_mean,score_5h_mean,score_6h_mean
0,6,2020-07-01 11:22:00,205.2,False,unknown,unknown,unknown,1,2,11,0,summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,2020-07-02 09:25:00,202.3,False,unknown,unknown,unknown,2,3,9,0,summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,2020-07-03 08:28:00,200.8,False,unknown,unknown,unknown,3,4,8,0,summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,2020-07-03 16:11:00,200.1,False,unknown,unknown,unknown,3,4,16,0,summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,2020-07-03 16:13:00,199.9,False,unknown,unknown,unknown,3,4,16,0,summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10982,80,2021-08-10 14:53:28,215.8,False,unknown,unknown,day,10,1,14,0,summer,3600.0,3000.0,3000.0,2250.0,2520.0,2700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,600.0,1350.0,1080.0,900.0,30.0,44.0,32.666667,25.5,21.4,24.000000
10983,80,2021-08-10 18:21:09,214.7,False,unknown,unknown,day,10,1,18,0,summer,3600.0,2700.0,2800.0,3000.0,3120.0,2900.0,0.0,0.0,200.0,150.0,120.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,900.0,600.0,450.0,360.0,600.0,4.0,6.0,6.666667,7.5,18.2,20.666667
10984,80,2021-08-11 01:06:33,216.1,False,unknown,unknown,day,11,2,1,0,summer,1800.0,2700.0,2200.0,2250.0,2400.0,2600.0,0.0,0.0,0.0,0.0,0.0,0.0,1200.0,600.0,400.0,300.0,240.0,200.0,600.0,300.0,1000.0,1050.0,960.0,800.0,0.0,4.5,27.333333,22.5,29.6,27.333333
10985,80,2021-08-11 01:13:28,212.7,False,unknown,unknown,night,11,2,1,0,summer,1800.0,2700.0,2200.0,2250.0,2400.0,2600.0,0.0,0.0,0.0,0.0,0.0,0.0,1200.0,600.0,400.0,300.0,240.0,200.0,600.0,300.0,1000.0,1050.0,960.0,800.0,0.0,4.5,27.333333,22.5,29.6,27.333333


In [18]:
weight_data['date'] = pd.to_datetime(weight_data['date'])
weight_data['Time'] = weight_data['date'].dt.time
weight_data['Date'] = weight_data['date'].dt.date

In [19]:
filtred_data_weight = weight_data.filter(['id', 'Date','Time','date', 'weight', 'WakeUp Time',	'toilet',	'meal',	'clothes'], axis=1)

In [20]:
filtred_data_weight

,id,Date,Time,date,weight,WakeUp Time,toilet,meal,clothes
0,6,2020-07-01,11:22:00,2020-07-01 11:22:00,205.2,False,unknown,unknown,unknown
1,6,2020-07-02,09:25:00,2020-07-02 09:25:00,202.3,False,unknown,unknown,unknown
2,6,2020-07-03,08:28:00,2020-07-03 08:28:00,200.8,False,unknown,unknown,unknown
3,6,2020-07-03,16:11:00,2020-07-03 16:11:00,200.1,False,unknown,unknown,unknown
4,6,2020-07-03,16:13:00,2020-07-03 16:13:00,199.9,False,unknown,unknown,unknown
...,...,...,...,...,...,...,...,...,...
10982,80,2021-08-10,14:53:28,2021-08-10 14:53:28,215.8,False,unknown,unknown,day
10983,80,2021-08-10,18:21:09,2021-08-10 18:21:09,214.7,False,unknown,unknown,day
10984,80,2021-08-11,01:06:33,2021-08-11 01:06:33,216.1,False,unknown,unknown,day
10985,80,2021-08-11,01:13:28,2021-08-11 01:13:28,212.7,False,unknown,unknown,night


In [21]:
import numpy as np
weight_data_reduced = (pd
     .DataFrame(filtred_data_weight)#.agg({'date' : [np.min, np.max]})
    .groupby(['id',	'Date'])
    .apply(lambda gdf: (gdf
    .assign(first_measurement=lambda first_measurement: filtred_data_weight['date'].min())))) 

In [22]:
weight_data_reduced.loc[:,~weight_data_reduced.columns.duplicated()]

id        Date  ...  clothes   first_measurement
id Date                              ...                             
6  2020-07-01 0       6  2020-07-01  ...  unknown 2019-07-06 09:35:00
   2020-07-02 1       6  2020-07-02  ...  unknown 2019-07-06 09:35:00
   2020-07-03 2       6  2020-07-03  ...  unknown 2019-07-06 09:35:00
              3       6  2020-07-03  ...  unknown 2019-07-06 09:35:00
              4       6  2020-07-03  ...  unknown 2019-07-06 09:35:00
...                  ..         ...  ...      ...                 ...
80 2021-08-10 10982  80  2021-08-10  ...      day 2019-07-06 09:35:00
              10983  80  2021-08-10  ...      day 2019-07-06 09:35:00
   2021-08-11 10984  80  2021-08-11  ...      day 2019-07-06 09:35:00
              10985  80  2021-08-11  ...    night 2019-07-06 09:35:00
              10986  80  2021-08-11  ...    night 2019-07-06 09:35:00

[10987 rows x 10 columns]

# Joining part 

in order to calculate the difference between wakeUp Time and first measurmenet 

In [23]:
filtred_data_weight

,id,Date,Time,date,weight,WakeUp Time,toilet,meal,clothes
0,6,2020-07-01,11:22:00,2020-07-01 11:22:00,205.2,False,unknown,unknown,unknown
1,6,2020-07-02,09:25:00,2020-07-02 09:25:00,202.3,False,unknown,unknown,unknown
2,6,2020-07-03,08:28:00,2020-07-03 08:28:00,200.8,False,unknown,unknown,unknown
3,6,2020-07-03,16:11:00,2020-07-03 16:11:00,200.1,False,unknown,unknown,unknown
4,6,2020-07-03,16:13:00,2020-07-03 16:13:00,199.9,False,unknown,unknown,unknown
...,...,...,...,...,...,...,...,...,...
10982,80,2021-08-10,14:53:28,2021-08-10 14:53:28,215.8,False,unknown,unknown,day
10983,80,2021-08-10,18:21:09,2021-08-10 18:21:09,214.7,False,unknown,unknown,day
10984,80,2021-08-11,01:06:33,2021-08-11 01:06:33,216.1,False,unknown,unknown,day
10985,80,2021-08-11,01:13:28,2021-08-11 01:13:28,212.7,False,unknown,unknown,night


In [24]:
activity_data

,id,date,status,loc,duration,sleep_duration,weekUpTime,Time,Date
0,6,2021-05-01 00:10:03,sleep,master bedroom,6743,0 days 01:52:23,2021-05-01 02:02:26,00:10:03,2021-05-01
1,6,2021-05-01 02:02:26,active,NaN,2255,NaT,NaT,02:02:26,2021-05-01
2,6,2021-05-01 02:40:01,sleep,master bedroom,5026,0 days 01:23:46,2021-05-01 04:03:47,02:40:01,2021-05-01
3,6,2021-05-01 04:03:47,active,NaN,2175,NaT,NaT,04:03:47,2021-05-01
4,6,2021-05-01 04:40:02,sleep,master bedroom,6888,0 days 01:54:48,2021-05-01 06:34:50,04:40:02,2021-05-01
...,...,...,...,...,...,...,...,...,...
15427,80,2021-08-18 15:25:02,still,kitchen,783,NaT,NaT,15:25:02,2021-08-18
15428,80,2021-08-18 15:38:05,active,NaN,4616,NaT,NaT,15:38:05,2021-08-18
15429,80,2021-08-18 16:55:01,still,kitchen,2233,NaT,NaT,16:55:01,2021-08-18
15430,80,2021-08-18 17:32:14,active,NaN,3168,NaT,NaT,17:32:14,2021-08-18


In [25]:
weight_activity_merged = pd.merge(filtred_data_weight[['id', 'Date', 'weight', 'toilet',	'meal',	'clothes']], activity_data, how='left', on= ['id', 'Date'])

In [26]:
weight_activity_merged

,id,Date,weight,toilet,meal,clothes,date,status,loc,duration,sleep_duration,weekUpTime,Time
0,6,2020-07-01,205.2,unknown,unknown,unknown,NaT,NaN,NaN,NaN,NaT,NaT,NaN
1,6,2020-07-02,202.3,unknown,unknown,unknown,NaT,NaN,NaN,NaN,NaT,NaT,NaN
2,6,2020-07-03,200.8,unknown,unknown,unknown,NaT,NaN,NaN,NaN,NaT,NaT,NaN
3,6,2020-07-03,200.1,unknown,unknown,unknown,NaT,NaN,NaN,NaN,NaT,NaT,NaN
4,6,2020-07-03,199.9,unknown,unknown,unknown,NaT,NaN,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73193,80,2021-08-11,212.1,unknown,unknown,night,2021-08-11 20:01:38,active,NaN,7703.0,NaT,NaT,20:01:38
73194,80,2021-08-11,212.1,unknown,unknown,night,2021-08-11 22:10:01,still,NaN,1445.0,NaT,NaT,22:10:01
73195,80,2021-08-11,212.1,unknown,unknown,night,2021-08-11 22:34:06,active,NaN,1255.0,NaT,NaT,22:34:06
73196,80,2021-08-11,212.1,unknown,unknown,night,2021-08-11 22:55:01,still,kitchen,2528.0,NaT,NaT,22:55:01


In [27]:
weight_activity_merged.query('weekUpTime != "NaT"')

,id,Date,weight,toilet,meal,clothes,date,status,loc,duration,sleep_duration,weekUpTime,Time
237,6,2021-05-17,194.6,unknown,unknown,night,2021-05-17 01:55:02,sleep,master bedroom,5385.0,0 days 01:29:45,2021-05-17 03:24:47,01:55:02
241,6,2021-05-17,194.6,unknown,unknown,night,2021-05-17 07:25:02,sleep,master bedroom,870.0,0 days 00:14:30,2021-05-17 07:39:32,07:25:02
263,6,2021-05-17,194.6,unknown,unknown,night,2021-05-17 23:10:01,sleep,master bedroom,9197.0,0 days 02:33:17,2021-05-18 01:43:18,23:10:01
265,6,2021-05-17,199.2,unknown,unknown,night,2021-05-17 01:55:02,sleep,master bedroom,5385.0,0 days 01:29:45,2021-05-17 03:24:47,01:55:02
269,6,2021-05-17,199.2,unknown,unknown,night,2021-05-17 07:25:02,sleep,master bedroom,870.0,0 days 00:14:30,2021-05-17 07:39:32,07:25:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73154,80,2021-08-11,212.7,unknown,unknown,night,2021-08-11 01:40:01,sleep,master bedroom,24260.0,0 days 06:44:20,2021-08-11 08:24:21,01:40:01
73164,80,2021-08-11,212.7,unknown,unknown,night,2021-08-11 15:25:02,sleep,master bedroom,156.0,0 days 00:02:36,2021-08-11 15:27:38,15:25:02
73174,80,2021-08-11,212.1,unknown,unknown,night,2021-08-11 00:10:01,sleep,master bedroom,1307.0,0 days 00:21:47,2021-08-11 00:31:48,00:10:01
73178,80,2021-08-11,212.1,unknown,unknown,night,2021-08-11 01:40:01,sleep,master bedroom,24260.0,0 days 06:44:20,2021-08-11 08:24:21,01:40:01


In [28]:
weight_activity_merged.rename(columns={'date':'weight_measurment_time'})

,id,Date,weight,toilet,meal,clothes,weight_measurment_time,status,loc,duration,sleep_duration,weekUpTime,Time
0,6,2020-07-01,205.2,unknown,unknown,unknown,NaT,NaN,NaN,NaN,NaT,NaT,NaN
1,6,2020-07-02,202.3,unknown,unknown,unknown,NaT,NaN,NaN,NaN,NaT,NaT,NaN
2,6,2020-07-03,200.8,unknown,unknown,unknown,NaT,NaN,NaN,NaN,NaT,NaT,NaN
3,6,2020-07-03,200.1,unknown,unknown,unknown,NaT,NaN,NaN,NaN,NaT,NaT,NaN
4,6,2020-07-03,199.9,unknown,unknown,unknown,NaT,NaN,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73193,80,2021-08-11,212.1,unknown,unknown,night,2021-08-11 20:01:38,active,NaN,7703.0,NaT,NaT,20:01:38
73194,80,2021-08-11,212.1,unknown,unknown,night,2021-08-11 22:10:01,still,NaN,1445.0,NaT,NaT,22:10:01
73195,80,2021-08-11,212.1,unknown,unknown,night,2021-08-11 22:34:06,active,NaN,1255.0,NaT,NaT,22:34:06
73196,80,2021-08-11,212.1,unknown,unknown,night,2021-08-11 22:55:01,still,kitchen,2528.0,NaT,NaT,22:55:01


In [30]:
weight_activity_merged['diff_weight_meas_weak'] = weight_activity_merged['weekUpTime'] - weight_activity_merged['date']

In [31]:
weight_activity_merged.to_csv('weight_activity_dataset_merged.csv')

In [29]:
import numpy as np
weight_activity_merged = (pd
     .DataFrame(weight_activity_merged)#.agg({'date' : [np.min, np.max]})
    .groupby(['id',	'Date'])
    .apply(lambda gdf: (gdf
    #.assign(first_measurement=lambda first_measurement: weight_activity_merged['date'].min())
    .assign(diff_weight_meas_weak = lambda diff_weight_meas_weak :weight_activity_merged['weekUpTime'] - weight_activity_merged['date'])
    )
)
) 

In [31]:
weight_activity_merged

id        Date  ...      Time diff_weight_meas_weak
id Date                              ...                                
6  2020-07-01 0       6  2020-07-01  ...       NaN                   NaT
   2020-07-02 1       6  2020-07-02  ...       NaN                   NaT
   2020-07-03 2       6  2020-07-03  ...       NaN                   NaT
              3       6  2020-07-03  ...       NaN                   NaT
              4       6  2020-07-03  ...       NaN                   NaT
...                  ..         ...  ...       ...                   ...
80 2021-08-11 73193  80  2021-08-11  ...  20:01:38                   NaT
              73194  80  2021-08-11  ...  22:10:01                   NaT
              73195  80  2021-08-11  ...  22:34:06                   NaT
              73196  80  2021-08-11  ...  22:55:01                   NaT
              73197  80  2021-08-11  ...  23:37:09                   NaT

[73198 rows x 14 columns]

In [ ]:
weight_activity_merged.query('weekUpTime != "NaT"')

id        Date  ...      Time diff_weight_meas_weak
id Date                              ...                                
6  2021-05-17 237     6  2021-05-17  ...  01:55:02       0 days 01:29:45
              241     6  2021-05-17  ...  07:25:02       0 days 00:14:30
              263     6  2021-05-17  ...  23:10:01       0 days 02:33:17
              265     6  2021-05-17  ...  01:55:02       0 days 01:29:45
              269     6  2021-05-17  ...  07:25:02       0 days 00:14:30
...                  ..         ...  ...       ...                   ...
80 2021-08-11 73154  80  2021-08-11  ...  01:40:01       0 days 06:44:20
              73164  80  2021-08-11  ...  15:25:02       0 days 00:02:36
              73174  80  2021-08-11  ...  00:10:01       0 days 00:21:47
              73178  80  2021-08-11  ...  01:40:01       0 days 06:44:20
              73188  80  2021-08-11  ...  15:25:02       0 days 00:02:36

[20893 rows x 14 columns]